# 1 Convolutional Neural networks
<p>We'll implement a Convolutional Neural network (CNN).</p>

<ul>
    <li><a href="#model">CNN</a></li>
    <li><a href="#model1">Determine size of the output</a></li>
    <li><a href="#concepts">Concepts: stride, zero padding</a></li>
</ul>

In [2]:
import torch 
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage, misc

/tmp/ipykernel_183235/3053585304.py:5: DeprecationWarning: scipy.misc is deprecated and will be removed in 2.0.0
  from scipy import ndimage, misc


## <a name="model" id="model">Model</a>

In [3]:
# 2D Convolution object
conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3)
conv

Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1))

In [5]:
# Set model values specifically
conv.state_dict()['weight'][0][0] = torch.tensor([[1.0,0,-1.0],[2.0,0,-2.0],[1.0,0.0,-1.0]])
conv.state_dict()['bias'][0] = 0.0
conv.state_dict()

OrderedDict([('weight',
              tensor([[[[ 1.,  0., -1.],
                        [ 2.,  0., -2.],
                        [ 1.,  0., -1.]]]])),
             ('bias', tensor([0.]))])

In [6]:
# Dummy input image tensor
# #inputs, #outputs, #rows, #columns
image = torch.zeros(1,1,5,5)
image[0,0,:,2] = 1
image

tensor([[[[0., 0., 1., 0., 0.],
          [0., 0., 1., 0., 0.],
          [0., 0., 1., 0., 0.],
          [0., 0., 1., 0., 0.],
          [0., 0., 1., 0., 0.]]]])

In [7]:
# Result of applying convolution (kernel element-level product and shifting) 
z = conv(image)
z

tensor([[[[-4.,  0.,  4.],
          [-4.,  0.,  4.],
          [-4.,  0.,  4.]]]], grad_fn=<ConvolutionBackward0>)

## <a name="model1" id="model1">Determine size of the output</a>

In [19]:
# Dummy square image input
M = 4
image1 = torch.ones(1, 1, M, M)
image1

tensor([[[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]]])

In [12]:
# In this experiment we assume square sized images. 
# Rectangular images would require each dimension to be treated independently.
# `O = M - K + 1`

# Define a kernel size = 2
K = 2
conv1 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=K)
conv1.state_dict()['weight'][0][0] = torch.tensor([ [1.0, 1.0], [1.0, 1.0] ])
conv1.state_dict()['bias'][0] = 0.0
conv1

Conv2d(1, 1, kernel_size=(2, 2), stride=(1, 1))

In [16]:
z1 = conv1(image1)
print("z1:",z1)
print("shape:",z1.shape[2:4])

z1: tensor([[[[4., 4., 4.],
          [4., 4., 4.],
          [4., 4., 4.]]]], grad_fn=<ConvolutionBackward0>)
shape: torch.Size([3, 3])


## <a name="concepts" id="concepts">Concepts: stride, zero padding</a>

### Stride

In [17]:
# Create a convolutional object with stride = 2
conv3 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=2, stride=2)

conv3.state_dict()['weight'][0][0]=torch.tensor([[1.0,1.0],[1.0,1.0]])
conv3.state_dict()['bias'][0]=0.0
conv3.state_dict()

OrderedDict([('weight',
              tensor([[[[1., 1.],
                        [1., 1.]]]])),
             ('bias', tensor([0.]))])

In [18]:
z3 = conv3(image1)

print("z3:", z3)
print("shape:", z3.shape[2:4])
# Observe the size of the output (activation map)

z3: tensor([[[[4., 4.],
          [4., 4.]]]], grad_fn=<ConvolutionBackward0>)
shape: torch.Size([2, 2])


### Padding

In [20]:
# Padding allows to keep the input at a reasonable size for kernel processing
image1

tensor([[[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]]])

In [21]:
# Perform conv with K=2 and stride=3
conv4 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=2, stride=3)
conv4.state_dict()['weight'][0][0]=torch.tensor([[1.0,1.0],[1.0,1.0]])
conv4.state_dict()['bias'][0]=0.0
conv4.state_dict()

z4 = conv4(image1)
print("z4:",z4)
print("z4:",z4.shape[2:4])

z4: tensor([[[[4.]]]], grad_fn=<ConvolutionBackward0>)
z4: torch.Size([1, 1])


In [22]:
# Zero-Padding
conv5 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=2, stride=3, padding=1)
conv5.state_dict()['weight'][0][0]=torch.tensor([[1.0,1.0], [1.0,1.0]])
conv5.state_dict()['bias'][0]=0.0
conv5.state_dict()

z5=conv5(image1)
print("z5:",z5)
print("z5:",z4.shape[2:4])

z5: tensor([[[[1., 2.],
          [2., 4.]]]], grad_fn=<ConvolutionBackward0>)
z5: torch.Size([1, 1])


### Determine the values of the output

In [23]:
image_test = torch.randn(1, 1, 4, 4)
image

tensor([[[[0., 0., 1., 0., 0.],
          [0., 0., 1., 0., 0.],
          [0., 0., 1., 0., 0.],
          [0., 0., 1., 0., 0.],
          [0., 0., 1., 0., 0.]]]])

If we apply a kernel of zeros K=3, to the previous image, the output is always zero.


In [24]:
conv_zeros = nn.Conv2d(in_channels=1, out_channels=1,kernel_size=3)
conv_zeros.state_dict()['weight'][0][0] = torch.tensor([[0,0,0],[0,0,0],[0,0.0,0]])
conv_zeros.state_dict()['bias'][0] = 0.0

In [25]:
z_zeros = conv_zeros(image_test)
print("z_zeros:", z_zeros)

z_zeros: tensor([[[[0., 0.],
          [0., 0.]]]], grad_fn=<ConvolutionBackward0>)
